In [0]:
from pyspark.sql import SparkSession, Row
# from test_helper import Test

In [0]:
# create HDFS directory
# $ hdfs dfs -mkdir -p /user/user

In [0]:
# move txt files to it
# $ hdfs dfs -put orders.csv

In [0]:
# 
spark = SparkSession.builder.master("local").appName("orders").getOrCreate()

In [0]:
# read the csv file
orders = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/michaelkennethkingston@gmail.com/orders.csv")

In [0]:
orders.take(5)

Out[5]: [Row(OrderDate='2009-04-05', ISBN='3437212490', Title='COOKING WITH MUSHROOMS', Category='COOKING', PriceEach='19.95', Quantity='1', FirstName='BONITA', LastName='MORALES', City='EASTPOINT'),
 Row(OrderDate='2009-04-05', ISBN='8843172113', Title='DATABASE IMPLEMENTATION', Category='COMPUTER', PriceEach='55.95', Quantity='1', FirstName='BONITA', LastName='MORALES', City='EASTPOINT'),
 Row(OrderDate='2009-04-01', ISBN='8843172113', Title='DATABASE IMPLEMENTATION', Category='COMPUTER', PriceEach='55.95', Quantity='1', FirstName='BONITA', LastName='MORALES', City='EASTPOINT'),
 Row(OrderDate='2009-04-01', ISBN='1059831198', Title='BODYBUILD IN 10 MINUTES A DAY', Category='FITNESS', PriceEach='30.95', Quantity='1', FirstName='BONITA', LastName='MORALES', City='EASTPOINT'),
 Row(OrderDate='2009-04-01', ISBN='3437212490', Title='COOKING WITH MUSHROOMS', Category='COOKING', PriceEach='19.95', Quantity='1', FirstName='BONITA', LastName='MORALES', City='EASTPOINT')]

In [0]:
# create a view to query
orders.createOrReplaceTempView("orders")

In [0]:
results = spark.sql('SELECT * from orders').show()

+----------+----------+--------------------+-----------+---------+--------+---------+--------+-----------+
| OrderDate|      ISBN|               Title|   Category|PriceEach|Quantity|FirstName|LastName|       City|
+----------+----------+--------------------+-----------+---------+--------+---------+--------+-----------+
|2009-04-05|3437212490|COOKING WITH MUSH...|    COOKING|    19.95|       1|   BONITA| MORALES|  EASTPOINT|
|2009-04-05|8843172113|DATABASE IMPLEMEN...|   COMPUTER|    55.95|       1|   BONITA| MORALES|  EASTPOINT|
|2009-04-01|8843172113|DATABASE IMPLEMEN...|   COMPUTER|    55.95|       1|   BONITA| MORALES|  EASTPOINT|
|2009-04-01|1059831198|BODYBUILD IN 10 M...|    FITNESS|    30.95|       1|   BONITA| MORALES|  EASTPOINT|
|2009-04-01|3437212490|COOKING WITH MUSH...|    COOKING|    19.95|       1|   BONITA| MORALES|  EASTPOINT|
|2009-04-04|2491748320|PAINLESS CHILD-RE...|FAMILY LIFE|    89.95|       1|    LEILA|   SMITH|TALLAHASSEE|
|2009-04-01|9959789321|E-BUSINESS THE

In [0]:
# query the view
results = spark.sql("SELECT OrderDate, count(Quantity) as qty FROM orders GROUP BY OrderDate")

In [0]:
# why isn't qty aggregating correctly?
# it looks like i need to return the query without a header
results.orderBy(results.qty.desc(), results.OrderDate.asc()).show()

+----------+---+
| OrderDate|qty|
+----------+---+
|2009-04-03|  9|
|2009-04-01|  6|
|2009-04-02|  5|
|2009-03-31|  4|
|2009-04-04|  4|
|2009-04-05|  4|
|2009-04-08|  1|
+----------+---+



In [0]:
# save to output
results.write.format("csv").save("dbfs:/FileStore/shared_uploads/michaelkennethkingston@gmail.com/result")

In [0]:
# 
spark.stop()